#### Este notebook aborda as etapas de Business Understanding e Data Understanding. As análises exploratórias e a engenharia de atributos e modelagem serão detalhadas nos notebooks subsequentes

# 1 SETUP

In [10]:
# Notebook Config 
import sys
import os
proj_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(proj_root)

# Data Analysis
import pandas as pd
import numpy as np

# DataViz 
import matplotlib.pyplot as plt
import seaborn as sns

# Statistical
from scipy.stats import chi2_contingency

# My Functions
from src.utils import df_summary_with_fraud, low_importance_numerical

# 2 BUSINESS UNDERSTANDING
***

O crescimento do comércio eletrônico e dos meios de pagamento digitais trouxe consigo uma expansão significativa nas tentativas de fraude em transações financeiras. Para instituições financeiras e empresas de meios de pagamento, a prevenção à fraude é um desafio central, pois envolve não apenas a proteção do patrimônio da instituição, mas também a confiança do cliente.

Um sistema de detecção de fraude precisa equilibrar dois objetivos conflitantes:

1. Reduzir perdas financeiras provenientes de transações fraudulentas;

2. Minimizar falsos positivos, ou seja, evitar que transações legítimas sejam erroneamente classificadas como suspeitas, o que compromete a experiência do cliente e pode gerar perda de receita.

No contexto deste trabalho, utiliza-se uma base de dados real disponibilizada pela competição IEEE-CIS Fraud Detection (Kaggle), que contém transações de e-commerce com informações de clientes, dispositivos, endereços e características da compra. O conjunto reflete a complexidade do problema enfrentado por instituições financeiras, com dados de alta dimensionalidade, presença de valores faltantes, variáveis anônimas e forte desbalanceamento da variável alvo (fraude).

O objetivo é desenvolver e avaliar técnicas de detecção de anomalias aplicadas à prevenção de fraudes, explorando abordagens que permitam identificar padrões irregulares no comportamento das transações. Essa análise é relevante para a área de negócios porque permite:

 - Antecipar tentativas de fraude com base em sinais sutis capturados pelas variáveis;

 - Reduzir perdas financeiras diretas associadas a transações fraudulentas aprovadas;

 - Preservar a experiência do cliente, reduzindo casos de bloqueio indevido;

 - Gerar insights para a política de segurança corporativa, apoiando decisões estratégicas sobre limites de risco e investimentos em inteligência antifraude.

Assim, o estudo conecta diretamente a aplicação de técnicas de ciência de dados com os objetivos estratégicos de uma instituição financeira: maximizar a segurança, preservar a confiança do cliente e aumentar a eficiência operacional dos sistemas antifraude.

# 3 DATA UNDERSTANDING
***

O dataset utilizado neste trabalho foi disponibilizado pela competição **IEEE-CIS Fraud Detection** (Kaggle), em parceria com a Vesta Corporation. Trata-se de uma base de dados real de transações de e-commerce, voltada especificamente para o desenvolvimento de modelos de **detecção de fraude em pagamentos digitais**.  

A base é composta por dois arquivos principais:  

- **`train_transaction.csv`** – Contém informações detalhadas sobre cada transação financeira, incluindo valores monetários, métodos de pagamento, variáveis derivadas de comportamento e características do comprador. É neste dataframe que se encontra a variável-alvo **`isFraud`**, indicando se a transação foi identificada como fraude (`1`) ou legítima (`0`).  
- **`train_identity.csv`** – Contém informações adicionais sobre identidade e dispositivo dos usuários em um subconjunto de transações. Inclui dados sobre endereços IP, navegadores, sistemas operacionais e características técnicas que podem auxiliar na identificação de comportamentos anômalos.  

Esses dois dataframes são conectados pela chave **`TransactionID`**, que funciona como identificador único de cada transação. Nem todas as transações possuem informações na tabela de identidade, refletindo um cenário real em que os dados disponíveis sobre o cliente variam conforme o contexto da compra.  

---

## Estrutura do dataset  

- **`train_transaction.csv`**: ~590 mil transações, com **394 variáveis**.  
- **`train_identity.csv`**: ~144 mil registros, com **41 variáveis**.  

---

## Dicionário resumido das principais variáveis  

| Variável         | Origem         | Tipo      | Descrição                                                                 |
|------------------|---------------|----------|---------------------------------------------------------------------------|
| `TransactionID`  | Transação     | Inteiro  | Identificador único da transação (chave primária).                        |
| `isFraud`        | Transação     | Binária  | Variável-alvo: indica se a transação foi fraude (`1`) ou legítima (`0`).  |
| `TransactionDT`  | Transação     | Inteiro  | Tempo da transação em relação a um ponto de referência (não é timestamp real). |
| `TransactionAmt` | Transação     | Numérica | Valor monetário da transação.                                             |
| `ProductCD`      | Transação     | Categórica | Categoria do produto/serviço adquirido.                                  |
| `card1`–`card6`  | Transação     | Numérica/Categórica | Atributos derivados de cartões de pagamento (parcialmente anonimizados). |
| `addr1`, `addr2` | Transação     | Numérica | Endereços vinculados ao comprador.                                        |
| `P_emaildomain`  | Transação     | Categórica | Domínio do e-mail do Pagador.                                          |
| `R_emaildomain`  | Transação     | Categórica | Domínio do e-mail do recebedor.                                          |
| `M1`–`M9`        | Transação     | Categórica (binária) | Flags de comportamento associadas às transações.                        |
| `DeviceType`     | Identidade    | Categórica | Tipo de dispositivo utilizado (ex.: desktop, mobile).                    |
| `DeviceInfo`     | Identidade    | Categórica | Informações adicionais sobre o dispositivo.                              |
| `id_12`–`id_38`  | Identidade    | Diversos | Variáveis de identidade anonimizadas (ex.: métodos de autenticação, sistema). |
| `V1`–`V339`      | Transação     | Numérica | Variáveis derivadas e anonimizadas, representam combinações internas de atributos originais. |

---


In [40]:
df_trans = pd.read_csv('../data/raw/train_transaction.csv')
df_id = pd.read_csv('../data/raw/train_identity.csv')

In [ ]:
# Prévia conteúdo Dataset transações

display(df_trans.head(10))

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2987002,0,86469,59.0,W,4663,490.0,150.0,visa,166.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2987003,0,86499,50.0,W,18132,567.0,150.0,mastercard,117.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2987004,0,86506,50.0,H,4497,514.0,150.0,mastercard,102.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,2987005,0,86510,49.0,W,5937,555.0,150.0,visa,226.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2987006,0,86522,159.0,W,12308,360.0,150.0,visa,166.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2987007,0,86529,422.5,W,12695,490.0,150.0,visa,226.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2987008,0,86535,15.0,H,2803,100.0,150.0,visa,226.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,2987009,0,86536,117.0,W,17399,111.0,150.0,mastercard,224.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Prévia conteúdo Dataset identidade

display(df_id.head(10))

,TransactionID,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,2987004,0.0,70787.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,samsung browser 6.2,32.0,2220x1080,match_status:2,T,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M
1,2987008,-5.0,98945.0,NaN,NaN,0.0,-5.0,NaN,NaN,NaN,...,mobile safari 11.0,32.0,1334x750,match_status:1,T,F,F,T,mobile,iOS Device
2,2987010,-5.0,191631.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,...,chrome 62.0,NaN,NaN,NaN,F,F,T,T,desktop,Windows
3,2987011,-5.0,221832.0,NaN,NaN,0.0,-6.0,NaN,NaN,NaN,...,chrome 62.0,NaN,NaN,NaN,F,F,T,T,desktop,NaN
4,2987016,0.0,7460.0,0.0,0.0,1.0,0.0,NaN,NaN,0.0,...,chrome 62.0,24.0,1280x800,match_status:2,T,F,T,T,desktop,MacOS
5,2987017,-5.0,61141.0,3.0,0.0,3.0,0.0,NaN,NaN,3.0,...,chrome 62.0,24.0,1366x768,match_status:2,T,F,T,T,desktop,Windows
6,2987022,-15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2987038,0.0,31964.0,0.0,0.0,0.0,-10.0,NaN,NaN,0.0,...,chrome 62.0,32.0,1920x1080,match_status:2,T,F,T,T,mobile,NaN
8,2987040,-10.0,116098.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,...,chrome 62.0,NaN,NaN,NaN,F,F,T,T,desktop,Windows
9,2987048,-5.0,257037.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,...,chrome 62.0,NaN,NaN,NaN,F,F,T,T,desktop,Windows


In [19]:
# Tamanho dos datasets 

print(f'Transactions: {df_trans.shape}')
print(f'Identity: {df_id.shape}')

Transactions: (590540, 394)
Identity: (144233, 41)


In [41]:
# Informações gerais dataset transações

print(df_trans.info(verbose=True))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 590540 entries, 0 to 590539
Data columns (total 394 columns):
 #    Column          Dtype  
---   ------          -----  
 0    TransactionID   int64  
 1    isFraud         int64  
 2    TransactionDT   int64  
 3    TransactionAmt  float64
 4    ProductCD       object 
 5    card1           int64  
 6    card2           float64
 7    card3           float64
 8    card4           object 
 9    card5           float64
 10   card6           object 
 11   addr1           float64
 12   addr2           float64
 13   dist1           float64
 14   dist2           float64
 15   P_emaildomain   object 
 16   R_emaildomain   object 
 17   C1              float64
 18   C2              float64
 19   C3              float64
 20   C4              float64
 21   C5              float64
 22   C6              float64
 23   C7              float64
 24   C8              float64
 25   C9              float64
 26   C10             float64
 27   C11         

In [37]:
# Informações gerais dataset identidade


print(df_id.info(verbose=True))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 144233 entries, 0 to 144232
Data columns (total 41 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   TransactionID  144233 non-null  int64  
 1   id_01          144233 non-null  float64
 2   id_02          140872 non-null  float64
 3   id_03          66324 non-null   float64
 4   id_04          66324 non-null   float64
 5   id_05          136865 non-null  float64
 6   id_06          136865 non-null  float64
 7   id_07          5155 non-null    float64
 8   id_08          5155 non-null    float64
 9   id_09          74926 non-null   float64
 10  id_10          74926 non-null   float64
 11  id_11          140978 non-null  float64
 12  id_12          144233 non-null  object 
 13  id_13          127320 non-null  float64
 14  id_14          80044 non-null   float64
 15  id_15          140985 non-null  object 
 16  id_16          129340 non-null  object 
 17  id_17          139369 non-nul

## 3.1 VARIÁVEIS DE CARTÃO
***

Nesta etapa, será avaliado o potencial das variáveis de cartão em representar o identificador do cliente (customer_id). Para isso, card1 será utilizada como proxy, possibilitando a detecção de anomalias em nível transacional de cartão.

In [ ]:
df_card = df_trans[['card1','card2','card3','card4','card5','card6','TransactionID','TransactionDT','isFraud']].sort_values(by=['card1','TransactionID'])
df_card

,card1,card2,card3,card4,card5,card6,TransactionID,TransactionDT,isFraud
243924,1000,555.0,185.0,mastercard,224.0,debit,3230924,5787419,0
36634,1001,555.0,150.0,visa,226.0,debit,3023634,916268,0
164336,1001,555.0,150.0,visa,226.0,debit,3151336,3504180,0
223739,1001,555.0,150.0,visa,226.0,debit,3210739,5270458,0
33767,1004,583.0,150.0,visa,226.0,credit,3020767,842821,0
...,...,...,...,...,...,...,...,...,...
432314,18395,543.0,150.0,mastercard,224.0,debit,3419314,10938204,0
433746,18395,543.0,150.0,mastercard,224.0,debit,3420746,10964952,0
459622,18395,543.0,150.0,mastercard,224.0,debit,3446622,11806909,0
493204,18395,543.0,150.0,mastercard,224.0,debit,3480204,12865537,0


Ao analisar as variáveis relacionadas ao cartão, observou-se que card1 representa o identificador único de cada cartão de crédito. Como não há uma variável explícita que identifique o cliente, card1 será utilizada como proxy de identificação, definindo a granularidade do dataframe final.

In [ ]:
# Verificar cartões que transacionaram apenas uma vez

unique_transaction = df_card.groupby('card1').size().reset_index(name='n_transacoes')
print(f'Cartões que transacionaram apenas uma vez: {len(unique_transaction[unique_transaction['n_transacoes'] == 1])}')

# Entender quantos desses cartões que transacionaram apenas uma vez e foram fraudes

unique_transaction = unique_transaction[unique_transaction['n_transacoes'] == 1]['card1'].to_list()
print(f'Cartões que transacionaram apenas uma vez e foram fraudes: {len(df_card[(df_card['card1'].isin(unique_transaction)) & (df_card['isFraud'] == 1)])}')

Cartões que transacionaram apenas uma vez: 3444
Cartões que transacionaram apenas uma vez e foram fraudes: 146


Verificou-se que 3.444 cartões (card1) realizaram apenas uma transação, representando 0,58% de todo o conjunto de dados. Dentre esses casos, 144 transações foram classificadas como fraude já no primeiro registro, correspondendo a 0,70% das fraudes totais. Essas observações isoladas representam um risco potencial de não serem detectadas como anomalias, uma vez que não possuem histórico transacional para comparação. Entretanto, considerando que esse grupo é numericamente reduzido e que futuramente podem ser desenvolvidas regras específicas para a detecção de fraudes sem histórico, a decisão de utilizar card1 como proxy de identificação do cliente e como base da granularidade do dataframe será mantida.

# RESULTADO FINAL

Após o entendimento do negócio e a análise preliminar dos dados, verificou-se que uma parcela significativa das variáveis encontrava-se anonimizada, o que comprometeria tanto a interpretação semântica quanto a análise de importância dessas variáveis para o modelo. Essa característica também dificultaria o processo de engenharia de atributos, uma vez que não seria possível compreender claramente o significado e a função de cada variável.

Diante desse cenário, definiu-se a variável `card1` como proxy para a identificação do cliente, alterando assim a granularidade do conjunto de dados para o nível transacional de cartão. A partir dessa decisão, todas as variáveis anonimizadas foram descartadas, mantendo-se apenas aquelas com descrição e interpretabilidade conhecidas. Essas variáveis serão exploradas na etapa de Análise Exploratória de Dados (EDA) e, posteriormente, utilizadas na engenharia de atributos (feature engineering) para a criação de novas variáveis com maior poder discriminatório.